In [ ]:
import os
import csv
from datasets import load_dataset
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from transformers import AutoProcessor, AutoModelForCausalLM
import requests
from PIL import Image

def img_caption(img):
    processor = AutoProcessor.from_pretrained("microsoft/git-base")
    model = AutoModelForCausalLM.from_pretrained("microsoft/git-base")
    image = Image.open(img)
    pixel_values = processor(images=image, return_tensors="pt").pixel_values
    generated_ids = model.generate(pixel_values=pixel_values, max_length=50)
    generated_caption = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
    return generated_caption

img_caption("./art_classifier/img/train/install/1516.jpg")

In [4]:

CAD_PATH = "/media/chunt/6F53-5578/CAD_img/"
TZVET_PATH = "/media/chunt/6F53-5578/TZVET_img/"

CAD_dir = [(CAD_PATH+f,f) for f in os.listdir(CAD_PATH) if os.path.isdir(os.path.join(CAD_PATH,f))]
TZVET_dir = [(TZVET_PATH+f,f) for f in os.listdir(TZVET_PATH) if os.path.isdir(os.path.join(TZVET_PATH,f))]

#def mass_create_dataset(dir):
    for d in dir:
        try:
            img_dir = d[0]
            label = d[1] if len(d[1]) < 40 else d[1][:40].rstrip("-")
            artist = label.split("-at-")[0].rstrip("-")
            data = [["file_name", "text"]]
            count = 0

            for filename in os.listdir(img_dir):
                if filename.endswith(".jpg"):
                    src = os.path.join(img_dir, filename)
                    caption = img_caption(src)
                    dst = os.path.join(img_dir, f"img{count}.jpg")
                    os.rename(src, dst)
                    data.append([f"img{count}.jpg", artist+", "+caption])
                    count += 1
                if filename.endswith(".pdf"):
                    os.remove(os.path.join(img_dir, filename))

            metadata_filename = img_dir + "/metadata.csv"
            with open(metadata_filename, mode='w', newline='') as file:
                writer = csv.writer(file)
                for row in data:
                    writer.writerow(row)
            repo_name = label if len(label) < 96 else label[:96]
            dataset = load_dataset("imagefolder", data_dir=img_dir, drop_labels=True)
            dataset.push_to_hub(f"Chunt0/{repo_name}", private=True)
            print(f"Directory {d} was uploaded")
        except:
            print(f"Directory {d} was skipped")

mass_create_dataset(TZVET_dir)

